In [1]:
from manim import *

config.media_width = "75%"
config.verbosity = "WARNING"

Manim Community v0.17.2

In [16]:
%%manim -qm ParetoFront

class ParetoFront(Scene):
    def get_rectangle_corners(self, bottom_left, top_right):
        return [
            (top_right[0], top_right[1]),
            (bottom_left[0], top_right[1]),
            (bottom_left[0], bottom_left[1]),
            (top_right[0], bottom_left[1]),
        ]


    def construct(self):
        ax = Axes(
            x_range=[0, 1, 0.1],
            y_range=[0, 1, 0.1],
            tips=False,
            axis_config={"include_numbers": True},
            # y_axis_config={"scaling": LogBase(custom_labels=True)},
        )

        labels = ax.get_axis_labels(
            Text("Objective 1").scale(0.5), Text("Objective 2").scale(0.5)
            )

        tx = ValueTracker(0.2)
        ty = ValueTracker(0.7)     

        def get_table():
            summary_table = Table(
            [[str(round(tx.get_value(),1)), str(round(ty.get_value(),1))]
            ],
            col_labels=[Text("Objective 1"), Text("Objective 2")],
            include_outer_lines=True,
            )
            summary_table.remove(*summary_table.get_vertical_lines()).scale(0.5).move_to(ax,RIGHT).align_to(ax,UP)
            return summary_table
        
        summary_table = always_redraw(get_table)
        
        pareto_front = ax.plot(lambda x: 1-x**2)
        area = ax.get_area(pareto_front, [0, 1], color=GREY, opacity=0.5)

        def get_rectangle():
            polygon = Polygon(
                *[
                    ax.c2p(*i)
                    for i in self.get_rectangle_corners(
                        (0, 0), (tx.get_value(), ty.get_value())
                    )
                ]
            )
            polygon.stroke_width = 1
            # polygon.set_fill(BLUE, opacity=0.5)
            polygon.set_stroke(YELLOW_B)
            return polygon

        polygon = always_redraw(get_rectangle)

        # initial_point = [ax.coords_to_point(tx.get_value(), ty.get_value())]
        dot = Dot()
        dot.add_updater(lambda x: x.move_to(ax.c2p(tx.get_value(), ty.get_value())))
        dot.set_z_index(10)
        # dot = Dot(point=initial_point, color=RED)  

        self.add(ax, labels, pareto_front, area, dot, summary_table)
        self.play(Create(polygon))
        self.play(ty.animate.set_value(0.96))
        self.play(tx.animate.set_value(0.2),ty.animate.set_value(0.7))
        self.play(tx.animate.set_value(0.5477))
        pareto_front.set_color(RED)
        self.play(Create(pareto_front))